# North Sea production visualization

This notebook contains code for cleaning data downloaded from the internet containing monthly production per field in UK and Norway side of North Sea. 

* UK: https://data-ogauthority.opendata.arcgis.com/datasets/oga-field-production-pprs-wgs84

* Norway: http://factpages.npd.no/factpages/Default.aspx?culture=en&nav1=field&nav2=TableView%7CProduction%7CSaleable%7CMonthly

The cleaned data is finally combined and visualized using the open-source geospatial data visualization web-service https://uber.github.io/kepler.gl/#/ 


The following resources have been also used:

* Find Latitude and Longitude of UK fields: https://en.wikipedia.org/wiki/List_of_oil_and_gas_fields_of_the_North_Sea#United_Kingdom

* Make videos: https://www.macworld.co.uk/how-to/mac-software/how-record-screen-on-your-mac-3527168/

* Convert to mpeg and GIF: http://www.convertfiles.com + https://cloudconvert.com/mpeg-to-gif + http://swift.logdown.com/posts/286503-tricks-and-tips-how-to-add-a-demo-video-in-your-git-readmemd-without-adding-files-to-the-repository-or-hosting-files-elsewhere-online

In [23]:
import os
import glob
from datetime import datetime,date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

# Figures inline and set visualization style
%matplotlib inline
sns.set()
sns.set_style("whitegrid")

## Data loading

In [24]:
df_UK = pd.read_csv('Data/UKCS_Production.csv', encoding='latin-1')
df_UK.fillna(0., inplace=True)

display(df_UK.head(5))

,OBJECTID,FIELDNAME,FIELDAREA,UNITNAME,UNITTYPCOD,UNITTYPDES,LOCATION,PERIODDATE,PERIODYRMN,PERIODMNTH,...,GCONDVOL,GCONDMBD,GCONDDEN,INJWATMASS,INJWATVOL,INJWATMBD,WATPRODMAS,WATPRODVOL,WATPRODMBD,ORGGRPNM
0,1,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-03-15T00:00:00.000Z,199403,3,...,0.0,0.0,0.0,74078.0,74078.0,15.030665,2192.0,2192.0,0.444764,CHEVRON CORPORATION
1,2,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-04-15T00:00:00.000Z,199404,4,...,0.0,0.0,0.0,217307.0,217307.0,45.562034,25516.0,25516.0,5.349855,CHEVRON CORPORATION
2,3,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-05-15T00:00:00.000Z,199405,5,...,0.0,0.0,0.0,334098.0,334098.0,67.789562,23496.0,23496.0,4.767414,CHEVRON CORPORATION
3,4,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-06-15T00:00:00.000Z,199406,6,...,0.0,0.0,0.0,297500.0,297500.0,62.375833,24628.0,24628.0,5.163671,CHEVRON CORPORATION
4,5,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-07-15T00:00:00.000Z,199407,7,...,0.0,0.0,0.0,321721.0,321721.0,65.278229,51591.0,51591.0,10.467980,CHEVRON CORPORATION


In [25]:
df_Nor = pd.read_csv('Data/NCS_Production.csv', encoding='utf-8')
#df_Nor.fillna(0., inplace=True)

display(df_Nor.head(5))

,prfInformationCarrier,prfYear,prfMonth,prfPrdOilGrossMillSm3,prfPrdGasGrossBillSm3,prfPrdCondensateGrossMillSm3,prfPrdOeGrossMillSm3,prfPrdProducedWaterInFieldMillSm3,prfNpdidInformationCarrier
0,33/9-6 DELTA,2009,7,0.00070,0.00011,0.0,0.00081,0.00051,44576
1,33/9-6 DELTA,2009,8,0.00292,0.00047,0.0,0.00339,0.00063,44576
2,33/9-6 DELTA,2009,9,0.00338,0.00054,0.0,0.00392,0.00316,44576
3,33/9-6 DELTA,2009,10,0.00312,0.00050,0.0,0.00362,0.00535,44576
4,33/9-6 DELTA,2009,11,0.00000,0.00000,0.0,0.00000,0.00000,44576


## Fields extraction

Extract all fields and save them to file

In [26]:
fields = df_UK['FIELDNAME'].unique()
print(fields)

#df_UK_position = pd.DataFrame({'FIELDNAME':fields, 
#                               'LAT':np.zeros(len(fields)),
#                               'LON':np.zeros(len(fields))})
#df_UK_position.to_csv('Data/Fields_UKCS.csv', encoding='latin-1')
#df_UK_position.to_excel('Data/Fields_UKCS.xls', encoding='latin-1')

[u'ALBA' u'AFFLECK' u'ANDREW' u'ALMA' u'ALDER' u'ANGUS' u'ARBROATH'
 u'ARGYLL' u'ARDMORE' u'AUK' u'ATHENA' u'ATLANTIC' u'ARKWRIGHT' u'BANFF'
 u'BEATRICE' u'BALMORAL' u'BARDOLINO' u'AUK NORTH' u'BACCHUS' u'AVIAT'
 u'BALLOCH' u'BEINN' u'BLAKE' u'BRAE-CENTRAL [Part of BRAE]' u'BIRCH'
 u'BITTERN' u'BEAULY' u'BLACKBIRD' u'BLADON' u'BLENHEIM' u'BLANE' u'BLAIR'
 u'BRAE-NORTH [Part of BRAE]' u'BRAE-SOUTH [Part of BRAE]' u'BRAEMAR'
 u'BRIMMOND' u'BRENDA' u'BRITANNIA' u'BRECHIN' u'BRODGAR' u'BURGHLEY'
 u'CALLANISH' u'CAPTAIN' u'CARNOUSTIE' u'BUCHAN' u'CALEDONIA' u'CHANTER'
 u'BUZZARD' u'BURGMAN' u'CATCHER' u'CAYLEY' u'CLAPHAM' u'CLAYMORE'
 u'CHESTNUT' u'CLYDE' u'CURLEW B + D' u'CROMARTY' u'COOK' u'CRATHES'
 u'DUNCAN' u'EAST BRAE' u'DONAN [MAERSK]' u'EGRET' u'CURLEW C' u'DRAKE'
 u'DUART' u'DAUNTLESS' u'DONAN' u'ELGIN' u'CYRUS' u'DURWARD' u'FIFE'
 u'FORTIES' u'EVEREST' u'FLORA' u'FERGUS' u'ERSKINE' u'ENOCHDHU'
 u'FLEMING' u'ENOCH' u'FLYNDRE' u'FARRAGON' u'ETTRICK' u'FULMAR'
 u'GADWALL' u'GANNET A'

In [27]:
df_Nor.rename(columns={'prfInformationCarrier': 'FIELDNAME'}, inplace=True)

fields = list(df_Nor['FIELDNAME'].unique())
print(fields)

#df_Nor_position = pd.DataFrame({'FIELDNAME':[field.encode('utf8') for field in fields], 
#                                'LAT':np.zeros(len(fields)), 
#                                'LON':np.zeros(len(fields))})
#df_Nor_position.to_csv('Data/Fields_NCS.csv', encoding='latin-1')

#df_Nor_position = pd.DataFrame({'FIELDNAME':fields, 
#                                'LAT':np.zeros(len(fields)), 
#                                'LON':np.zeros(len(fields))})
#df_Nor_position.to_excel('Data/Fields_NCS.xls', encoding='latin-1')

[u'33/9-6 DELTA', u'ALBUSKJELL', u'ALVE', u'ALVHEIM', u'ATLA', u'BALDER', u'BLANE', u'BRAGE', u'BRYNHILD', u'BYRDING', u'B\xd8YLA', u'COD', u'DRAUGEN', u'EDDA', u'EDVARD GRIEG', u'EKOFISK', u'ELDFISK', u'EMBLA', u'ENOCH', u'FLYNDRE', u'FRAM', u'FRAM H-NORD', u'FRIGG', u'FR\xd8Y', u'GAUPE', u'GIMLE', u'GINA KROG', u'GJ\xd8A', u'GLITNE', u'GOLIAT', u'GRANE', u'GUDRUN', u'GULLFAKS', u'GULLFAKS S\xd8R', u'GUNGNE', u'GYDA', u'HEIDRUN', u'HEIMDAL', u'HOD', u'HULDRA', u'HYME', u'ISLAY', u'IVAR AASEN', u'JETTE', u'JOTUN', u'KNARR', u'KRISTIN', u'KVITEBJ\xd8RN', u'LILLE-FRIGG', u'MARIA', u'MARULK', u'MIKKEL', u'MIME', u'MORVIN', u'MURCHISON', u'NJORD', u'NORD\xd8ST FRIGG', u'NORNE', u'ODIN', u'ORMEN LANGE', u'OSEBERG', u'OSEBERG S\xd8R', u'OSEBERG \xd8ST', u'OSELVAR', u'REV', u'RINGHORNE \xd8ST', u'SIGYN', u'SINDRE', u'SKARV', u'SKIRNE', u'SKULD', u'SLEIPNER VEST', u'SLEIPNER \xd8ST', u'SNORRE', u'SN\xd8HVIT', u'STATFJORD', u'STATFJORD NORD', u'STATFJORD \xd8ST', u'SVALIN', u'SYGNA', u'TAMBAR',

Read file with fields and location (lat-long) and join to main dataframe

In [28]:
df_UK_position = pd.read_excel('Data/Fields_UKCS.xls', index_col=0, encoding='latin-1')
df_UK          = df_UK.merge(df_UK_position)

df_Nor_position = pd.read_excel('Data/Fields_NCS.xls', index_col=0, encoding='latin-1')
df_Nor          = df_Nor.merge(df_Nor_position)

# get only data that have latitute
df_UK  = df_UK[df_UK.LAT!=0]
df_Nor = df_Nor[df_Nor.LAT!=0]

print(df_UK.FIELDNAME.unique())
print(df_Nor.FIELDNAME.unique())

display(df_UK.head(5))
display(df_Nor.head(5))

[u'ALBA' u'ANDREW' u'AUK' u'BRAE-CENTRAL [Part of BRAE]' u'BLANE'
 u'BRAE-NORTH [Part of BRAE]' u'BRAE-SOUTH [Part of BRAE]' u'BUCHAN'
 u'DRAKE' u'ELGIN' u'CYRUS' u'FORTIES' u'EVEREST' u'FLEMING' u'FARRAGON'
 u'FULMAR' u'FRANKLIN' u'GOLDEN EAGLE' u'HAWKINS' u'MILLER' u'SEYMOUR'
 u'HARDING' u'KINNOULL' u'AMETHYST WEST' u'ANGLIA'
 u'LEMAN [PERENCO][pt. of LEMAN]' u'CLAIR-PHASE 1 [Part of CLAIR]']
[u'BLANE' u'BRAGE' u'DRAUGEN' u'EKOFISK' u'FRIGG' u'GINA KROG' u'GOLIAT'
 u'GRANE' u'GULLFAKS' u'HEIDRUN' u'HEIMDAL' u'NORNE' u'ORMEN LANGE'
 u'OSEBERG' u'OSEBERG S\xd8R' u'OSEBERG \xd8ST' u'OSELVAR' u'SKIRNE'
 u'SLEIPNER VEST' u'SLEIPNER \xd8ST' u'SNORRE' u'SN\xd8HVIT' u'STATFJORD'
 u'STATFJORD NORD' u'STATFJORD \xd8ST' u'TAMBAR' u'TROLL' u'ULA' u'VALE'
 u'VALHALL' u'YME']


,OBJECTID,FIELDNAME,FIELDAREA,UNITNAME,UNITTYPCOD,UNITTYPDES,LOCATION,PERIODDATE,PERIODYRMN,PERIODMNTH,...,GCONDDEN,INJWATMASS,INJWATVOL,INJWATMBD,WATPRODMAS,WATPRODVOL,WATPRODMBD,ORGGRPNM,LAT,LON
0,1,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-03-15T00:00:00.000Z,199403,3,...,0.0,74078.0,74078.0,15.030665,2192.0,2192.0,0.444764,CHEVRON CORPORATION,58.05803,1.079633
1,2,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-04-15T00:00:00.000Z,199404,4,...,0.0,217307.0,217307.0,45.562034,25516.0,25516.0,5.349855,CHEVRON CORPORATION,58.05803,1.079633
2,3,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-05-15T00:00:00.000Z,199405,5,...,0.0,334098.0,334098.0,67.789562,23496.0,23496.0,4.767414,CHEVRON CORPORATION,58.05803,1.079633
3,4,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-06-15T00:00:00.000Z,199406,6,...,0.0,297500.0,297500.0,62.375833,24628.0,24628.0,5.163671,CHEVRON CORPORATION,58.05803,1.079633
4,5,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-07-15T00:00:00.000Z,199407,7,...,0.0,321721.0,321721.0,65.278229,51591.0,51591.0,10.467980,CHEVRON CORPORATION,58.05803,1.079633


,FIELDNAME,prfYear,prfMonth,prfPrdOilGrossMillSm3,prfPrdGasGrossBillSm3,prfPrdCondensateGrossMillSm3,prfPrdOeGrossMillSm3,prfPrdProducedWaterInFieldMillSm3,prfNpdidInformationCarrier,LAT,LON
810,BLANE,2007,5,0.0000,0.00000,0.0,0.00000,0.0000,3437650,56.847683,2.498706
811,BLANE,2007,6,0.0000,0.00000,0.0,0.00000,0.0000,3437650,56.847683,2.498706
812,BLANE,2007,7,0.0000,0.00000,0.0,0.00000,0.0000,3437650,56.847683,2.498706
813,BLANE,2007,8,0.0000,0.00000,0.0,0.00000,0.0000,3437650,56.847683,2.498706
814,BLANE,2007,9,0.0082,0.00048,0.0,0.00867,0.0001,3437650,56.847683,2.498706


## Data cleaning

Change dates in UK (move day to 1)

In [29]:
def change_day_in_date(df):
    return df.PERIODDATE.replace(day=1)

df_UK.PERIODDATE = pd.to_datetime(df_UK.PERIODDATE)
df_UK.PERIODDATE = df_UK.apply(change_day_in_date, axis=1)

Create date column in Norway data

In [30]:
def create_date(df):
    return datetime(df['prfYear'],df['prfMonth'],1)
    
df_Nor['PERIODDATE'] = df_Nor.apply(create_date, axis=1)

Choose only needec columns and merge two datasets

In [31]:
df_UK = df_UK[['FIELDNAME', 'LAT', 'LON', 'PERIODDATE', 'OILPRODM3', 'AGASPROMMS', 'WATPRODVOL']]

df_UK.rename(columns={'OILPRODM3': 'Oil Production Volumes', 
                      'AGASPROMMS': 'Gas Production Volumes',
                      'WATPRODVOL': 'Water Production Volumes'}, inplace=True)
display(df_UK.head(5))

df_UK.to_csv('Data/UKCS_Production_cleaned.csv', index=False, encoding='latin-1')

,FIELDNAME,LAT,LON,PERIODDATE,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
0,ALBA,58.05803,1.079633,1994-03-01,142134.0,6.691623,2192.0
1,ALBA,58.05803,1.079633,1994-04-01,166535.0,9.130105,25516.0
2,ALBA,58.05803,1.079633,1994-05-01,160681.0,9.311768,23496.0
3,ALBA,58.05803,1.079633,1994-06-01,198771.0,9.996499,24628.0
4,ALBA,58.05803,1.079633,1994-07-01,248810.0,12.571001,51591.0


In [32]:
df_Nor = df_Nor[['FIELDNAME', 'LAT', 'LON', 'PERIODDATE', 'prfPrdOilGrossMillSm3', 
                 'prfPrdGasGrossBillSm3', 'prfPrdProducedWaterInFieldMillSm3']]

df_Nor.rename(columns={'prfPrdOilGrossMillSm3': 'Oil Production Volumes', 
                       'prfPrdGasGrossBillSm3': 'Gas Production Volumes',
                       'prfPrdProducedWaterInFieldMillSm3': 'Water Production Volumes'}, 
              inplace=True)

# change units
df_Nor['Oil Production Volumes']= df_Nor['Oil Production Volumes']*1e6 
df_Nor['Gas Production Volumes']= df_Nor['Gas Production Volumes']*1e6 
df_Nor['Water Production Volumes']= df_Nor['Water Production Volumes']*1e6 

display(df_Nor.head(5))

df_Nor.to_csv('Data/NCS_Production_cleaned.csv', index=False, encoding='latin-1')

,FIELDNAME,LAT,LON,PERIODDATE,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
810,BLANE,56.847683,2.498706,2007-05-01,0.0,0.0,0.0
811,BLANE,56.847683,2.498706,2007-06-01,0.0,0.0,0.0
812,BLANE,56.847683,2.498706,2007-07-01,0.0,0.0,0.0
813,BLANE,56.847683,2.498706,2007-08-01,0.0,0.0,0.0
814,BLANE,56.847683,2.498706,2007-09-01,8200.0,480.0,100.0


In [33]:
df_Nor_total = df_Nor.groupby(by='FIELDNAME').sum()

In [34]:
df_UK_total  = df_UK.groupby(by='FIELDNAME').sum()

Fields with biggest production

In [35]:
df_UK_total.sort_values(by='Oil Production Volumes', ascending=False)

,LAT,LON,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
FIELDNAME,,,,,
FORTIES,29377.783503,517.483503,4.400745e+08,28944.333749,5.059658e+08
FULMAR,24317.280000,1209.600000,9.035153e+07,14201.625847,1.066534e+08
ALBA,16836.828700,313.093570,6.755544e+07,4039.592545,2.374325e+08
MILLER,11867.500000,268.660000,5.472607e+07,20996.820555,2.070433e+07
HARDING,15556.450000,341.900000,4.390850e+07,3945.562919,1.119725e+08
ELGIN,11591.300000,406.000000,4.387846e+07,29920.604498,1.255588e+06
BRAE-SOUTH [Part of BRAE],24419.200000,540.758400,4.208048e+07,16189.435763,2.698923e+07
FRANKLIN,11248.897000,395.970000,2.965870e+07,39308.050533,1.516597e+06
ANDREW,15151.050000,300.150000,2.690404e+07,11060.498423,5.028720e+07


In [36]:
df_Nor_total.sort_values(by='Oil Production Volumes', ascending=False)

,LAT,LON,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
FIELDNAME,,,,,
STATFJORD,28191.072000,696.432700,573433950.0,184878580.0,515603420.0
EKOFISK,31655.049582,1755.460390,485898060.0,203353970.0,174075700.0
OSEBERG,22851.073924,945.162722,397911630.0,273300020.0,46074280.0
GULLFAKS,22983.142880,813.134968,365547290.0,103022300.0,386953440.0
TROLL,20558.843484,1263.245871,265567770.0,634390410.0,356066750.0
SNORRE,18884.452664,654.235736,215353140.0,39266200.0,148498180.0
HEIDRUN,17602.191000,1942.110000,158700370.0,44662570.0,75521210.0
DRAUGEN,18878.075748,2194.926972,139795880.0,7744420.0,117805700.0
VALHALL,23925.923231,1377.819962,119418560.0,26616030.0,9038510.0


In [37]:
df_UK_total[df_UK_total['Oil Production Volumes']==df_UK_total['Oil Production Volumes'].max()]

,LAT,LON,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
FIELDNAME,,,,,
FORTIES,29377.783503,517.483503,440074530.0,28944.333749,505965839.0


In [38]:
df_Nor_total[df_Nor_total['Oil Production Volumes']==df_Nor_total['Oil Production Volumes'].max()]

,LAT,LON,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
FIELDNAME,,,,,
STATFJORD,28191.072,696.4327,573433950.0,184878580.0,515603420.0


Merge production for common fields (fields that partially belong to UK and partially to Norway)

In [39]:
display((df_Nor[df_Nor.FIELDNAME=='BLANE']).sort_values(by='PERIODDATE', axis=0, ascending=True).head(10))
display((df_UK[df_UK.FIELDNAME=='BLANE']).sort_values(by='PERIODDATE', axis=0, ascending=True).head(10))

,FIELDNAME,LAT,LON,PERIODDATE,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
810,BLANE,56.847683,2.498706,2007-05-01,0.0,0.0,0.0
811,BLANE,56.847683,2.498706,2007-06-01,0.0,0.0,0.0
812,BLANE,56.847683,2.498706,2007-07-01,0.0,0.0,0.0
813,BLANE,56.847683,2.498706,2007-08-01,0.0,0.0,0.0
814,BLANE,56.847683,2.498706,2007-09-01,8200.0,480.0,100.0
815,BLANE,56.847683,2.498706,2007-10-01,14350.0,850.0,70.0
816,BLANE,56.847683,2.498706,2007-11-01,13750.0,810.0,50.0
817,BLANE,56.847683,2.498706,2007-12-01,14700.0,870.0,40.0
818,BLANE,56.847683,2.498706,2008-01-01,14070.0,860.0,50.0
819,BLANE,56.847683,2.498706,2008-02-01,12850.0,740.0,40.0


,FIELDNAME,LAT,LON,PERIODDATE,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
5270,BLANE,56.847683,2.498706,2007-09-01,37337.06,2.556053,436.24
5271,BLANE,56.847683,2.498706,2007-10-01,65381.88,4.399793,336.20
5272,BLANE,56.847683,2.498706,2007-11-01,62657.02,4.326370,205.82
5273,BLANE,56.847683,2.498706,2007-12-01,66979.24,4.531507,193.52
5274,BLANE,56.847683,2.498706,2008-01-01,64104.32,4.445566,222.22
5275,BLANE,56.847683,2.498706,2008-02-01,58524.22,4.102093,179.58
5276,BLANE,56.847683,2.498706,2008-03-01,67100.60,4.448369,201.72
5277,BLANE,56.847683,2.498706,2008-04-01,36124.28,2.412226,109.06
5278,BLANE,56.847683,2.498706,2008-05-01,67671.32,4.397925,186.96
5279,BLANE,56.847683,2.498706,2008-06-01,56847.32,3.823461,164.00


In [40]:
common_fields = list(set(df_UK.FIELDNAME.unique()).intersection(df_Nor.FIELDNAME.unique()))

for common_field in common_fields:
    print('Working with {}...'.format(common_field))

    df_merged=pd.merge(df_Nor[df_Nor.FIELDNAME==common_field],df_UK[df_UK.FIELDNAME==common_field], 
                       on='PERIODDATE', how='outer', suffixes=('', '_UK'))
    df_merged.fillna(0., inplace=True)

    df_merged['Oil Production Volumes']   = df_merged['Oil Production Volumes']   + df_merged['Oil Production Volumes_UK'] 
    df_merged['Gas Production Volumes']   = df_merged['Gas Production Volumes']   + df_merged['Gas Production Volumes_UK'] 
    df_merged['Water Production Volumes'] = df_merged['Water Production Volumes'] + df_merged['Water Production Volumes_UK']
    df_merged = df_merged[df_Nor.columns]
    
    df_Nor = df_Nor[df_Nor.FIELDNAME!=common_field]
    df_UK  = df_UK[df_UK.FIELDNAME!=common_field]

    df_Nor = pd.concat([df_Nor, df_merged], ignore_index=True)

Working with BLANE...


In [41]:
display((df_Nor[df_Nor.FIELDNAME=='BLANE']).sort_values(by='PERIODDATE', axis=0, ascending=True).head(10))

,FIELDNAME,LAT,LON,PERIODDATE,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
7780,BLANE,56.847683,2.498706,2007-05-01,0.00,0.000000,0.00
7781,BLANE,56.847683,2.498706,2007-06-01,0.00,0.000000,0.00
7782,BLANE,56.847683,2.498706,2007-07-01,0.00,0.000000,0.00
7783,BLANE,56.847683,2.498706,2007-08-01,0.00,0.000000,0.00
7784,BLANE,56.847683,2.498706,2007-09-01,45537.06,482.556053,536.24
7785,BLANE,56.847683,2.498706,2007-10-01,79731.88,854.399793,406.20
7786,BLANE,56.847683,2.498706,2007-11-01,76407.02,814.326370,255.82
7787,BLANE,56.847683,2.498706,2007-12-01,81679.24,874.531507,233.52
7788,BLANE,56.847683,2.498706,2008-01-01,78174.32,864.445566,272.22
7789,BLANE,56.847683,2.498706,2008-02-01,71374.22,744.102093,219.58


Finally merge datasets

In [42]:
df_NCS = pd.concat([df_Nor,df_UK],axis=0).reset_index(drop=True)
display(df_NCS)

,FIELDNAME,LAT,LON,PERIODDATE,Oil Production Volumes,Gas Production Volumes,Water Production Volumes
0,BRAGE,60.58,3.02,1993-09-01,30810.0,1840.000000,0.0
1,BRAGE,60.58,3.02,1993-10-01,231810.0,12040.000000,0.0
2,BRAGE,60.58,3.02,1993-11-01,374290.0,21470.000000,0.0
3,BRAGE,60.58,3.02,1993-12-01,420840.0,26090.000000,0.0
4,BRAGE,60.58,3.02,1994-01-01,414550.0,24880.000000,0.0
5,BRAGE,60.58,3.02,1994-02-01,388400.0,24570.000000,0.0
6,BRAGE,60.58,3.02,1994-03-01,423790.0,27460.000000,0.0
7,BRAGE,60.58,3.02,1994-04-01,409210.0,26440.000000,0.0
8,BRAGE,60.58,3.02,1994-05-01,467480.0,29370.000000,0.0
9,BRAGE,60.58,3.02,1994-06-01,464500.0,28920.000000,0.0


## Additional features 

Modify data to include column with cumulative production per field

In [43]:
df_NCS['Cumulative Oil Production Volumes']   = np.zeros(len(df_NCS))
df_NCS['Cumulative Gas Production Volumes']   = np.zeros(len(df_NCS))
df_NCS['Cumulative Water Production Volumes'] = np.zeros(len(df_NCS))

for field in df_NCS.FIELDNAME.unique():
    print('Working with {}...'.format(field.encode('utf-8')))
    df_field = df_NCS[df_NCS['FIELDNAME']==field]

    df_field = df_field.drop(labels='FIELDNAME', axis=1)        
    df_field.PERIODDATE = pd.to_datetime(df_field.PERIODDATE)
    
    df_field_cumulative = df_field.sort_values(by='PERIODDATE',
                                               axis=0, ascending=True)[['Oil Production Volumes',
                                                                        'Gas Production Volumes',
                                                                        'Water Production Volumes']].cumsum(axis=0)
    #display(df_field_cumulative.head(10))

    df_field_cumulative.rename(columns={'Oil Production Volumes': 'Cumulative Oil Production Volumes', 
                                        'Gas Production Volumes': 'Cumulative Gas Production Volumes',
                                        'Water Production Volumes': 'Cumulative Water Production Volumes'}, inplace=True)
    #display(df_field_cumulative.head(10))
    
    df_NCS.loc[df_NCS['FIELDNAME']==field,'Cumulative Oil Production Volumes'] = \
        df_field_cumulative['Cumulative Oil Production Volumes']
    
    df_NCS.loc[df_NCS['FIELDNAME']==field,'Cumulative Gas Production Volumes'] = \
        df_field_cumulative['Cumulative Gas Production Volumes']
   
    df_NCS.loc[df_NCS['FIELDNAME']==field,'Cumulative Water Production Volumes'] = \
        df_field_cumulative['Cumulative Water Production Volumes']
    
display(df_NCS)

Working with BRAGE...
Working with DRAUGEN...
Working with EKOFISK...
Working with FRIGG...
Working with GINA KROG...
Working with GOLIAT...
Working with GRANE...
Working with GULLFAKS...
Working with HEIDRUN...
Working with HEIMDAL...
Working with NORNE...
Working with ORMEN LANGE...
Working with OSEBERG...
Working with OSEBERG SØR...
Working with OSEBERG ØST...
Working with OSELVAR...
Working with SKIRNE...
Working with SLEIPNER VEST...
Working with SLEIPNER ØST...
Working with SNORRE...
Working with SNØHVIT...
Working with STATFJORD...
Working with STATFJORD NORD...
Working with STATFJORD ØST...
Working with TAMBAR...
Working with TROLL...
Working with ULA...
Working with VALE...
Working with VALHALL...
Working with YME...
Working with BLANE...
Working with ALBA...
Working with ANDREW...
Working with AUK...
Working with BRAE-CENTRAL [Part of BRAE]...
Working with BRAE-NORTH [Part of BRAE]...
Working with BRAE-SOUTH [Part of BRAE]...
Working with BUCHAN...
Working with DRAKE...
Worki

,FIELDNAME,LAT,LON,PERIODDATE,Oil Production Volumes,Gas Production Volumes,Water Production Volumes,Cumulative Oil Production Volumes,Cumulative Gas Production Volumes,Cumulative Water Production Volumes
0,BRAGE,60.58,3.02,1993-09-01,30810.0,1840.000000,0.0,30810.0,1.840000e+03,0.0
1,BRAGE,60.58,3.02,1993-10-01,231810.0,12040.000000,0.0,262620.0,1.388000e+04,0.0
2,BRAGE,60.58,3.02,1993-11-01,374290.0,21470.000000,0.0,636910.0,3.535000e+04,0.0
3,BRAGE,60.58,3.02,1993-12-01,420840.0,26090.000000,0.0,1057750.0,6.144000e+04,0.0
4,BRAGE,60.58,3.02,1994-01-01,414550.0,24880.000000,0.0,1472300.0,8.632000e+04,0.0
5,BRAGE,60.58,3.02,1994-02-01,388400.0,24570.000000,0.0,1860700.0,1.108900e+05,0.0
6,BRAGE,60.58,3.02,1994-03-01,423790.0,27460.000000,0.0,2284490.0,1.383500e+05,0.0
7,BRAGE,60.58,3.02,1994-04-01,409210.0,26440.000000,0.0,2693700.0,1.647900e+05,0.0
8,BRAGE,60.58,3.02,1994-05-01,467480.0,29370.000000,0.0,3161180.0,1.941600e+05,0.0
9,BRAGE,60.58,3.02,1994-06-01,464500.0,28920.000000,0.0,3625680.0,2.230800e+05,0.0


Save final dataset

In [44]:
df_NCS.PERIODDATE = df_NCS.PERIODDATE.dt.strftime('%Y-%m-%dT00:00:00.000Z')

df_NCS.to_csv('Data/NorthSea_Production_cleaned.csv', index=False, encoding='latin-1')